
# Pipeline ANOVA SVM

This example shows how a feature selection can be easily integrated within
a machine learning pipeline.

We also show that you can easily inspect part of the pipeline.


We will start by generating a binary classification dataset. Subsequently, we
will divide the dataset into two subsets.



In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(
    n_features=20,
    n_informative=3,
    n_redundant=0,
    n_classes=2,
    n_clusters_per_class=2,
    random_state=42,
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

A common mistake done with feature selection is to search a subset of
discriminative features on the full dataset, instead of only using the
training set. The usage of scikit-learn :func:`~sklearn.pipeline.Pipeline`
prevents to make such mistake.

Here, we will demonstrate how to build a pipeline where the first step will
be the feature selection.

When calling `fit` on the training data, a subset of feature will be selected
and the index of these selected features will be stored. The feature selector
will subsequently reduce the number of features, and pass this subset to the
classifier which will be trained.



# Explanation of SelectKBest

`SelectKBest` is a feature selection method in scikit-learn that selects the top `k` features based on a specified scoring function. It helps in reducing the dimensionality of the feature space, improving model performance and interpretability.

## How SelectKBest Works

### Arguments:
- `score_func`: The scoring function to evaluate the importance of features. Common options are:
  - `f_classif`: ANOVA F-value between label/feature for classification tasks.
  - `chi2`: Chi-squared statistic between label/feature for classification tasks.
  - `mutual_info_classif`: Mutual information for a discrete target.
  - `f_regression`: F-value between label/feature for regression tasks.
  - `mutual_info_regression`: Mutual information for a continuous target.
- `k`: Number of top features to select. Can be an integer or `"all"` to select all features.

### Math Behind SelectKBest:
- **f_classif (ANOVA F-value)**:
  - Computes the ANOVA F-value for each feature.
  - The F-value measures the ratio of variance between the groups to the variance within the groups.
  - A higher F-value indicates that the feature is more likely to be significant.

The formula for F-value in ANOVA:
$\ F = \frac{MSR}{MSE} $
where:
- $\ MSR $ is the mean square regression (variance between the groups).
- $\ MSE $ is the mean square error (variance within the groups).

### Steps in Feature Selection with SelectKBest:
1. **Compute the score**: For each feature, the scoring function (e.g., `f_classif`) computes a score that indicates the significance of the feature.
2. **Rank the features**: Features are ranked based on their scores.
3. **Select top k features**: The top `k` features with the highest scores are selected.

### Example Usage:
```python
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

# Select top 3 features using ANOVA F-value
anova_filter = SelectKBest(f_classif, k=3)
clf = LinearSVC()
anova_svm = make_pipeline(anova_filter, clf)
anova_svm.fit(X_train, y_train)


In [6]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

anova_filter = SelectKBest(f_classif, k=7)
clf = LinearSVC()
anova_svm = make_pipeline(anova_filter, clf)
anova_svm.fit(X_train, y_train)

c:\Users\marti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Pipeline(steps=[('selectkbest', SelectKBest(k=7)), ('linearsvc', LinearSVC())])

Once the training is complete, we can predict on new unseen samples. In this
case, the feature selector will only select the most discriminative features
based on the information stored during training. Then, the data will be
passed to the classifier which will make the prediction.

Here, we show the final metrics via a classification report.



In [7]:
from sklearn.metrics import classification_report

y_pred = anova_svm.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.73      0.85        15
           1       0.71      1.00      0.83        10

    accuracy                           0.84        25
   macro avg       0.86      0.87      0.84        25
weighted avg       0.89      0.84      0.84        25



Be aware that you can inspect a step in the pipeline. For instance, we might
be interested about the parameters of the classifier. Since we selected
three features, we expect to have three coefficients.



In [8]:
anova_svm[-1].coef_

array([[0.20854111, 0.9276843 , 0.1472149 , 0.28023679, 0.16733161,
        0.22735165, 0.29925922]])

However, we do not know which features were selected from the original
dataset. We could proceed by several manners. Here, we will invert the
transformation of these coefficients to get information about the original
space.



In [9]:
anova_svm[:-1].inverse_transform(anova_svm[-1].coef_)

array([[0.        , 0.20854111, 0.9276843 , 0.        , 0.        ,
        0.1472149 , 0.        , 0.        , 0.        , 0.28023679,
        0.        , 0.        , 0.16733161, 0.        , 0.        ,
        0.22735165, 0.        , 0.        , 0.        , 0.29925922]])

We can see that the features with non-zero coefficients are the selected
features by the first step.

